
# Data Processing
<br>


### Import libraries

In [1]:
# nltk
from nltk import pos_tag
from nltk import RegexpTokenizer, PorterStemmer, WordNetLemmatizer, FreqDist
from nltk.corpus import stopwords

# utilities
import string
import numpy as np
import pandas as pd

# sklearn
from sklearn.preprocessing import LabelEncoder

# preferences
pd.set_option('display.max_columns', None)


### Read in dataset, preview first three rows, and display column and datatype info
Dataset downloaded from https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt. The full dataset includes 130M+ customer reviews, grouped by product category, from 1995 to 2015. Each row represents one review.

Set the path variable to the local directory where the (unzipped) dataset resides. Other datasets may be processed by modifying the filename.

In [2]:
# read amazon tsv dataset into pandas df
path = 'data/'                               #<---local directory path where dataset resides
filename = 'amazon_reviews_us_LUGGAGE_v1_00' #<---name of dataset being processed
ext = '.tsv'                                 #<---file extension (tab seperated values)

cols = ['marketplace',       #<---2 letter country code of review marketplace
        'customer_id',       #<---random identifier to aggregate reviews by single author
        'review_id',         #<---unique ID of review
        'product_id',        #<---unique ID of product to which review pertains
        'product_parent',    #<---random identifier to aggregate reviews for same product
        'product_title',     #<---product title
        'product_category',  #<---product category to group dataset into coherent parts 
        'star_rating',       #<---1-5 star rating of product
        'helpful_votes',     #<---number of helpful votes review received
        'total_votes',       #<---total number of votes review received
        'vine',              #<---review part of Vine program
        'verified_purchase', #<---review of verified purchase
        'review_headline',   #<---review title
        'review_body',       #<---review text
        'review_date']       #<---review date

df = pd.read_csv(path+filename+ext,
                 sep='\t',
                 usecols = cols)

display(df.head(3))
df.info()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,40884699,R9CO86UUJCAW5,B00VGTN02Y,786681372,Teenage Mutant Ninja Turtle Boys' Teenage Muta...,Luggage,3.0,0.0,0.0,N,Y,my review of this product was in error. It ...,my review of this product was in error. It was...,2015-08-31
1,US,23208852,R3PR8X6QGVJ8B1,B005KIWL0E,618251799,"Kenneth Cole Reaction Out of Bounds 20"" 4 Whe...",Luggage,5.0,0.0,0.0,N,Y,Five Stars,Perfect size.,2015-08-31
2,US,17100246,R39BO2819ABUPF,B007UNSHJ6,810480328,American Tourister Luggage AT Pop 3 Piece Spin...,Luggage,4.0,0.0,0.0,N,Y,so good.,"So far, so good.",2015-08-31


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348474 entries, 0 to 348473
Data columns (total 15 columns):
marketplace          348474 non-null object
customer_id          348474 non-null int64
review_id            348474 non-null object
product_id           348474 non-null object
product_parent       348474 non-null int64
product_title        348474 non-null object
product_category     348474 non-null object
star_rating          348473 non-null float64
helpful_votes        348473 non-null float64
total_votes          348473 non-null float64
vine                 348473 non-null object
verified_purchase    348473 non-null object
review_headline      348468 non-null object
review_body          348452 non-null object
review_date          348472 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 39.9+ MB



### Drop junk data, tokenize text, and drop stopwords

Reviews with fewer than 10 votes 

In [3]:
# drop reviews with null data, fewer than 10 votes, or duplicate text
df = df.dropna()
df = df.loc[df.total_votes >= 10]
df = df.drop_duplicates(subset=['review_body'])
df = df.reset_index(drop=True)

# tokenize text and define and remove stopwords from tokens
df['tokens'] = df.review_body.apply(RegexpTokenizer(r'[a-zA-Z0-9]+').tokenize)

stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)

df['stopped_tokens'] = df.tokens.apply(
    lambda x: [word.lower() for word in x if word.lower() not in stopwords_list])


### Extract features and define target label as above/below median helpful/unhelpful ratio

In [4]:
# lexical features
df['LEM']  = df.stopped_tokens.apply(lambda x: [WordNetLemmatizer().lemmatize(word) for word in x])
df['STEM'] = df.stopped_tokens.apply(lambda x: [PorterStemmer().stem(word) for word in x]) #<---lemm/stem stopped tokens (normalize vocabulary

# syntactic features
df['POS']  = df.stopped_tokens.apply(lambda x: [pos_tag[1] for pos_tag in pos_tag(x)])     #<---pos-tagged tokens ("bag of tags")
df['NOUN'] = df.POS.apply(
    lambda x: sum(1 for pos in x if pos.startswith('NN')) / len(x) if len(x) > 0 else 0)   #<---percentage of nouns
df['ADJ']  = df.POS.apply(
    lambda x: sum(1 for pos in x if pos.startswith('JJ')) / len(x) if len(x) > 0 else 0)   #<---percentage of adjectives
df['ADV']  = df.POS.apply(
    lambda x: sum(1 for pos in x if pos.startswith('RB')) / len(x) if len(x) > 0 else 0)   #<---percentage of adverbs
df['VERB'] = df.POS.apply(
    lambda x: sum(1 for pos in x if pos.startswith('VB')) / len(x) if len(x) > 0 else 0)   #<---percentage of verbs

# structual features
df['CHAR']    = df.review_body.apply(lambda x: len(x))             #<---number of characters
df['NUM']     = df.stopped_tokens.apply(lambda x: len(x))          #<---number of tokens
df['WORD']    = df.review_body.apply(lambda x: len(x.split(' ')))  #<---number of words
df['SENT']    = df.review_body.apply(lambda x: len(x.split('. '))) #<---number of sentences
df['INTERRO'] = df.review_body.apply(lambda x: len(x.split('? '))) #<---number of questionss
df['EXCLAM']  = df.review_body.apply(lambda x: len(x.split('! '))) #<---number of exclamations
df['COUNT']   = df.review_body.str.count('!')                      #<---number of exclamation points
df['LEN']     = df.CHAR / df.WORD                                  #<---average word length
df['AVG']     = df.WORD / df.SENT                                  #<---average sentence length
df['PER']     = df.INTERRO / df.SENT                               #<---percentage of questions
df['CAPS']    = df.review_body.apply(
    lambda x: len([char for char in x if char.isupper()==True]) 
              / len(x))                                            #<---percentage of capitalized characters

# con-textual features
df['STAR']  = df.star_rating.astype(int)                                     #<---reviewer's star rating for product
df['MED']   = df.groupby('product_parent').STAR.transform('median')          #<---product's median star rating
df['FAV']   = df.STAR - df.MED                                               #<---reviewer's rating vs product's median
df['POP']   = df.groupby('product_parent').product_parent.transform('count') #<---number of product's reviews  
df['DATE']  = pd.to_datetime(df.review_date, errors='coerce',
                            yearfirst=True, infer_datetime_format=True)      #<---date of review(as datetime)
df['FIRST'] = df.groupby('product_parent').DATE.transform('min')             #<---date of product's first review 
df['DAYS']  = (df.DATE - df.FIRST).apply(lambda x: x.days)                   #<---days from review date to first review (int)

# define target as above/below median ratio of helpful votes to total votes (binary classifier)
df['HELP']   = df.helpful_votes / df.total_votes                 #<---percentage of helpful votes
df['TARGET'] = np.where(df.HELP > df.HELP.quantile(q=0.5), 1, 0) #<---TARGET threshold (0.5=median)


### Drop non-tokenized/non-quantitative data and POS tags, review first three rows, display column and datatype info, and write datafraome to csv

In [5]:
# keep target and extracted features, drop HELP (leakage) and TIME and FIRST (datetime)
cols = ['TARGET',  #<---ratio of helpful votes to total votes above/below median
        'LEM',     #<---lemmas
        'STEM',    #<---stems
        'NOUN',    #<---percentage of nouns
        'ADJ',     #<---percentage of adjectives
        'ADV',     #<---percentage of adverbs
        'VERB',    #<---percentage of verbs
        'CHAR',    #<---number of characters
        'NUM',     #<---number of tokens
        'WORD',    #<---number of words
        'SENT',    #<---number of sentences
        'INTERRO', #<---number of questionss
        'EXCLAM',  #<---number of exclamations
        'COUNT',   #<---number of exclamation points
        'LEN',     #<---average word length
        'AVG',     #<---average sentence length
        'PER',     #<---percentage of questions
        'CAPS',    #<---percentage of capitalized characters
        'STAR',    #<---reviewer's star rating for product
        'MED',     #<---product's median star rating
        'FAV',     #<---reviewer's star rating vs product's median
        'POP',     #<---number of product's reviews
        'DAYS']    #<---days from review date to first review

# drop original features and any nan values from feature extraction
df = df[cols].dropna()

display(df.head(3))
df.info()

df.to_csv(path+filename+'_processed'+ext, index=0)

,TARGET,LEM,STEM,NOUN,ADJ,ADV,VERB,CHAR,NUM,WORD,SENT,INTERRO,EXCLAM,COUNT,LEN,AVG,PER,CAPS,STAR,MED,FAV,POP,DAYS
0,1,"[strap, broke, supposed, anti, theft, strap, b...","[strap, broke, suppos, anti, theft, strap, bro...",0.440000,0.160000,0.080000,0.200000,318,25,63,6,1,2,3,5.047619,10.5,0.166667,0.031447,1,4.0,-3.0,23,2001
1,0,"[absolutely, thrilled, quality, leather, bette...","[absolut, thrill, qualiti, leather, better, ex...",0.166667,0.250000,0.416667,0.166667,136,12,26,1,1,3,3,5.230769,26.0,1.000000,0.029412,5,5.0,0.0,2,191
2,0,"[picking, luggage, 2, week, whirlwind, tour, e...","[pick, luggag, 2, week, whirlwind, tour, europ...",0.370968,0.177419,0.096774,0.209677,728,62,140,8,1,2,2,5.200000,17.5,0.125000,0.020604,5,5.0,0.0,3,98


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16242 entries, 0 to 16241
Data columns (total 23 columns):
TARGET     16242 non-null int64
LEM        16242 non-null object
STEM       16242 non-null object
NOUN       16242 non-null float64
ADJ        16242 non-null float64
ADV        16242 non-null float64
VERB       16242 non-null float64
CHAR       16242 non-null int64
NUM        16242 non-null int64
WORD       16242 non-null int64
SENT       16242 non-null int64
INTERRO    16242 non-null int64
EXCLAM     16242 non-null int64
COUNT      16242 non-null int64
LEN        16242 non-null float64
AVG        16242 non-null float64
PER        16242 non-null float64
CAPS       16242 non-null float64
STAR       16242 non-null int64
MED        16242 non-null float64
FAV        16242 non-null float64
POP        16242 non-null int64
DAYS       16242 non-null int64
dtypes: float64(10), int64(11), object(2)
memory usage: 3.0+ MB
